In [37]:
import sched, time
import yfinance as yf
import cryptocompare as cp

In [7]:
import requests as r, json
from getpass import getpass

In [5]:
url = 'https://bigdataforall.com/crypto/?'

In [8]:
print('Enter your bitcoin account key')
key = getpass()

Enter your bitcoin account key
········


In [73]:
profile_url = f"{url}key={key}&request=profile"
reset_url =  f"{url}key={key}&request=reset"
buy_url = f"{url}key={key}&request=buy&symbol={sym}&quantity={qty}"
sell_url = f"{url}key={key}&request=sell&symbol={sym}&quantity={qty}"

In [70]:
## reset data
response = r.get(reset_url).json()
## set initial params
qty = 0.1  ## qty planning to buy 
sym = "BTC" 
p_qty = 0 ## purchased qty

In [71]:
response

{'username': 'jodie',
 'cash': 100000,
 'shares value': 0,
 'penalty': 0,
 'total': 100000,
 'shares': []}

In [72]:

while True:
    print ("checking balance and bitcoin price...")
    ## get the current BTC price and current account profile
    BTC = cp.get_price('BTC',currency='USD')['BTC']['USD']
    response = r.get(profile_url).json()
    cash = response['cash']
    total = response['total']
    b_asset = total - cash
    
    print(f"Cash = {cash} , Total = {total}, purchased bitcoin value = {b_asset} /// Bitcion price = {BTC}")
    
    if p_qty == 0: ## first time buy -- buy 20%
        qty = cash/5/BTC
        response = r.get(buy_url).json()
        p_qty = qty
        b_asset = cash/5
        print(f"purchased {qty} coins, now total own {p_qty} coins")
        
    elif avg_cost < BTC: ## if making money: sell 25% owned bitcoin
        qty = b_asset/4/BTC
        response = r.get(sell_url).json()
        p_qty = p_qty - qty
        print(f"sold {qty} coins, now total own {p_qty} coins")
        
    elif avg_cost < BTC:  ## if not making money: buy 12.5% cash value bitcoin
        qty = cash/8/BTC
        response = r.get(buy_url).json()
        p_qty = p_qty + qty
        b_asset = b_asset + (cash/8)
        print(f"purchased {qty} coins, now total own {p_qty} coins")
        
    else:
        print("no action")
        
    avg_cost = b_asset / p_qty
        
    
    
    time.sleep(25)

checking balance and bitcoin price...
Cash = 100000 , Total = 100000, purchased bitcoin value = 0 /// Bitcion price = 54928.47
purchased 0.36410990511842034 coins, now total own 0.36410990511842034 coins
checking balance and bitcoin price...
Cash = 94510.155 , Total = 100003.971, purchased bitcoin value = 5493.816000000006 /// Bitcion price = 54931.45
sold 0.02500305380615297 coins, now total own 0.33910685131226737 coins
checking balance and bitcoin price...
Cash = 99999.999 , Total = 99999.999, purchased bitcoin value = 0.0 /// Bitcion price = 54931.59
sold 0.0 coins, now total own 0.33910685131226737 coins
checking balance and bitcoin price...
Cash = 98999.999 , Total = 98999.999, purchased bitcoin value = 0.0 /// Bitcion price = 54953.98
sold 0.0 coins, now total own 0.33910685131226737 coins


KeyboardInterrupt: 

In [46]:
# test -- 
BTC = cp.get_price('BTC',currency='USD')['BTC']['USD']

In [47]:
BTC

54691.59